# Regression

In [1]:
import pandas as pd
import numpy  as np
import datetime

import tensorflow as tf
import xgboost    as xgb
from tensorflow       import keras
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics  import mean_absolute_error, median_absolute_error

import seaborn as sns

Let's start by loading the dataset and parsing the date.

In [2]:
border_crossing = pd.read_csv('../data/Border_Crossing_Entry_Data.csv')    
border_crossing.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524)
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207)
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333)
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996)
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005)


In [3]:
def parse_date(s): 
    date, *_ = s.strip().split()
    [month, day, year] = list(map(int, date.split(sep='/')))
    return datetime.date(year, month, day)
    
border_crossing['Date'] = border_crossing['Date'].apply(parse_date)
border_crossing = border_crossing.sort_values(by='Date')
border_crossing.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
346732,Presidio,Texas,2403,US-Mexico Border,1996-01-01,Trucks,347,POINT (-104.39000000000001 29.56)
345903,Whitlash,Montana,3321,US-Canada Border,1996-01-01,Truck Containers Full,0,POINT (-111.26000000000002 49)
345904,Jackman,Maine,104,US-Canada Border,1996-01-01,Truck Containers Full,2103,POINT (-70.4 45.81)
345905,Hidalgo,Texas,2305,US-Mexico Border,1996-01-01,Truck Containers Full,9794,POINT (-98.27 26.1)
345906,Boundary,Washington,3015,US-Canada Border,1996-01-01,Pedestrians,0,POINT (-117.62999999999998 49)


Now, we'll extract the year, month, and also keep a running count of the total number of months that have passed.

In [4]:
border_crossing['Year'] = border_crossing['Date'].apply(lambda d: d.year - 1996)
border_crossing['Month'] = border_crossing['Date'].apply(lambda d: d.month - 1)

def get_total_month(p):
    (y,m) = p
    return 12*y + m
border_crossing['Total Month'] = list(map(get_total_month, zip(border_crossing['Year'], border_crossing['Month'])))

border_crossing.tail()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location,Year,Month,Total Month
527,Calexico,California,2503,US-Mexico Border,2019-03-01,Personal Vehicles,413457,POINT (-115.49806000000001 32.67889),23,2,278
526,Richford,Vermont,203,US-Canada Border,2019-03-01,Train Passengers,30,POINT (-72.67832000000001 44.98588),23,2,278
525,Eastport,Idaho,3302,US-Canada Border,2019-03-01,Personal Vehicle Passengers,9719,POINT (-116.18027999999998 48.99944),23,2,278
523,Laurier,Washington,3016,US-Canada Border,2019-03-01,Rail Containers Full,196,POINT (-118.22302 48.99892),23,2,278
0,Calexico East,California,2507,US-Mexico Border,2019-03-01,Trucks,34447,POINT (-115.48433000000001 32.67524),23,2,278


The last thing we need to do to prepare the data is to one-hot encode the categorical data.

In [5]:
del border_crossing['Location']
one_hot = border_crossing.copy()

for col in ['Port Name', 'State', 'Port Code', 'Border', 'Measure']:
    border_crossing[col] = border_crossing[col].astype('category')

def one_hot_encode_categoricals(df):
    df2 = df.copy()
    for col in df.columns:
        if df[col].dtype.name == 'category':
            dummies = pd.get_dummies(df[col], prefix=col)
            df2[dummies.columns] = dummies
            del df2[col]
    return df2
    
one_hot = one_hot_encode_categoricals(border_crossing)
one_hot.head()

,Date,Value,Year,Month,Total Month,Port Name_Alcan,Port Name_Alexandria Bay,Port Name_Algonac,Port Name_Ambrose,Port Name_Anacortes,...,Measure_Pedestrians,Measure_Personal Vehicle Passengers,Measure_Personal Vehicles,Measure_Rail Containers Empty,Measure_Rail Containers Full,Measure_Train Passengers,Measure_Trains,Measure_Truck Containers Empty,Measure_Truck Containers Full,Measure_Trucks
346732,1996-01-01,347,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
345903,1996-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
345904,1996-01-01,2103,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
345905,1996-01-01,9794,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
345906,1996-01-01,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [6]:
def standardize(col):
    max = col.max()
    min = col.min()
    return (col - min)/(max - min)

def split_data(df, label_col, split=.9):
    data = df.copy()
    del data[label_col]
    labels = df[label_col]
    
    len_total = len(df)
    len_train = int(.9 * len_total)
    len_test  = len_total - len_train
    
    train_input  = data.head(len_train).to_numpy()
    train_labels = labels.head(len_train).to_numpy()

    test_input  = data.tail(len_test).to_numpy()
    test_labels = labels.tail(len_test).to_numpy()
    
    return train_input, train_labels, test_input, test_labels

shuffled = one_hot.sample(frac=1)
del shuffled['Date']
train_input, train_labels, test_input, test_labels = split_data(shuffled, 'Value')

Let's start with the random forest algorithm.

In [7]:
forest = RandomForestRegressor(n_estimators=100, max_depth=100, random_state=731)

In [8]:
forest.fit(train_input, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=731, verbose=0,
                      warm_start=False)

In [9]:
print(forest.score(test_input, test_labels))
print(mean_absolute_error(test_labels, forest.predict(test_input)))

0.9892847122288758
1875.048866978013


To put this error into context, let's look at the distribution:

In [10]:
border_crossing['Value'].describe()

count    3.467330e+05
mean     2.818767e+04
std      1.518588e+05
min      0.000000e+00
25%      0.000000e+00
50%      9.000000e+01
75%      2.483000e+03
max      4.447374e+06
Name: Value, dtype: float64

As we can see, most data is very small with respect to the maximum. So now I am curious if the model is consistently off, or if it strays infrequently, but severely.

In [11]:
median_absolute_error(test_labels, forest.predict(test_input))

15.104999999999997

This suggests to me that the error is typically very small, but modest mistakes on the extremely large values greatly contribute to the average error.

Now let's try XGBoost.

In [12]:
dtrain = xgb.DMatrix(train_input, label=train_labels)
dtest = xgb.DMatrix(test_input, label=test_labels)

param = {'max_depth':100, 'eta':1, 'num_parallel_tree':10,
         'verbosity':1, 'objective':'reg:squarederror', 'eval_metric':'mae' }
num_round = 3
bst = xgb.train(param, dtrain, num_round)

In [13]:
predictions = bst.predict(dtest)

print(mean_absolute_error(test_labels, predictions))
print(median_absolute_error(test_labels, predictions))

2201.4189274093706
22.954687118530273


Surprisingly, this is not as succesful as Random Forest.

Now let's try to apply a neural network.

In [14]:
len_hidden = len(border_crossing.columns)

nn = tf.keras.Sequential([
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(1)
])

nn.compile(loss = 'mse',
           optimizer = tf.keras.optimizers.RMSprop(0.001),
           metrics=['mae','mse']
          )

In [15]:
nn.fit(train_input, train_labels, epochs=10, verbose=0)

In [16]:
print(nn.evaluate(test_input, test_labels, verbose=0))
print(median_absolute_error(test_labels, nn.predict(test_input)))

[8209199965.550672, 26280.27, 8209203000.0]
9655.727783203125


These results aren't very encouraging. Let's try again after standardizing our data.

In [17]:
shuffled = one_hot.sample(frac=1)
del shuffled['Date']
shuffled['Value'] = standardize(shuffled['Value'])
    
train_input, train_labels, test_input, test_labels = split_data(shuffled, 'Value')

In [18]:
nn = tf.keras.Sequential([
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(1)
])

nn.compile(loss = 'mse',
           optimizer = tf.keras.optimizers.RMSprop(0.001),
           metrics=['mae','mse']
          )

In [19]:
nn.fit(train_input, train_labels, epochs=10, verbose=0)

In [20]:
print(nn.evaluate(test_input, test_labels, verbose=0))
print(median_absolute_error(test_labels, nn.predict(test_input)))

[8.136535817736513e-05, 0.0020646658, 8.13653e-05]
0.0006404444575309753


How does this compare? What are the values after scaling back to the original range?

In [37]:
max = border_crossing['Value'].max()
print(.002064 * max)
print(.000640 * max)

9179.379936
2846.3193600000004


The values appear to be slightly better. Now, let's just tweak some parameters of the neural network itself, such as the depth, loss function, and optimizer.

In [22]:
nn = tf.keras.Sequential([
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(1)
])

nn.compile(loss = 'mse',
           optimizer = tf.keras.optimizers.RMSprop(0.001),
           metrics=['mae','mse']
          )

In [23]:
nn.fit(train_input, train_labels, epochs=10, verbose=0)

In [24]:
print(nn.evaluate(test_input, test_labels, verbose=0))
print(median_absolute_error(test_labels, nn.predict(test_input)))

[5.2944052583408995e-05, 0.003127423, 5.2944066e-05]
0.0021943261054550383


In [38]:
max = border_crossing['Value'].max()
print(.003127 * max)
print(.002194 * max)

13906.938498
9757.538556000001


Now let's try changing the loss function. I think mean absolute error might be better here than squared error, which will over-punish predictions on large numbers that are relatively accurate.

In [26]:
nn = tf.keras.Sequential([
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(1)
])

nn.compile(loss = 'mae',
           optimizer = tf.keras.optimizers.RMSprop(0.001),
           metrics=['mae','mse']
          )

In [27]:
nn.fit(train_input, train_labels, epochs=10, verbose=0)

In [28]:
print(nn.evaluate(test_input, test_labels, verbose=0))
print(median_absolute_error(test_labels, nn.predict(test_input)))

[0.00591330693763362, 0.0059133046, 0.0010180448]
3.9642618503421545e-05


Finally, let's change the optimizer. I found "adam" to be the most successful.

In [29]:
nn = tf.keras.Sequential([
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(1)
])

nn.compile(loss = 'mae',
           optimizer = 'adam',
           metrics=['mse']
          )

In [30]:
nn.fit(train_input, train_labels, epochs=10, verbose=0)

In [31]:
print(nn.evaluate(test_input, test_labels, verbose=0))
print(median_absolute_error(test_labels, nn.predict(test_input)))

[0.0014434797453620773, 4.9045128e-05]
9.927981992243505e-05


In [39]:
max = border_crossing['Value'].max()
print(.00144348 * max)
print(.00009928 * max)

6419.69542152
441.53529072


In [33]:
nn = tf.keras.Sequential([
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(len_hidden, activation='relu'),
    keras.layers.Dense(1)
])

nn.compile(loss = 'mae',
           optimizer = 'adam',
           metrics=['mse']
          )

In [34]:
nn.fit(train_input, train_labels, epochs=25, verbose=0)

In [35]:
print(nn.evaluate(test_input, test_labels, verbose=0))
print(median_absolute_error(test_labels, nn.predict(test_input)))

[0.001088601034192307, 4.6762194e-05]
8.467078441753983e-05


In [40]:
max = border_crossing['Value'].max()
print(.00108860 * max)
print(.00008467 * max)

4841.4113363999995
376.55915658


At this point, I am unable to improve accuracy with the neural network. It is very suprising to me that random forest was the most accurate, as opposed to the neural network. Perhaps this was a result of the unusual value distribution.